In [1727]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [1728]:
#reading csv
content = pd.read_csv("Housing.csv")

In [1729]:
#displaying information to see what is inside 
content.head()

,price,area,bedrooms,bathrooms,stories,mainroad,guestroom,basement,hotwaterheating,airconditioning,parking,prefarea,furnishingstatus
0,13300000,7420,4,2,3,yes,no,no,no,yes,2,yes,furnished
1,12250000,8960,4,4,4,yes,no,no,no,yes,3,no,furnished
2,12250000,9960,3,2,2,yes,no,yes,no,no,2,yes,semi-furnished
3,12215000,7500,4,2,2,yes,no,yes,no,yes,3,yes,furnished
4,11410000,7420,4,1,2,yes,yes,yes,no,yes,2,no,furnished


In [1730]:
#choosing only neccesary columns from original DataFrame
content = content[["price", "area", "bedrooms","bathrooms",]]
norm = content.copy()
norm.head()

,price,area,bedrooms,bathrooms
0,13300000,7420,4,2
1,12250000,8960,4,4
2,12250000,9960,3,2
3,12215000,7500,4,2
4,11410000,7420,4,1


In [1731]:
#types checking to be sure they are numbers not text
norm.dtypes

price        int64
area         int64
bedrooms     int64
bathrooms    int64
dtype: object

In [1732]:
#Preparing data 
columns = norm.columns.tolist()
for column in columns[1:]:
        norm[column] = (content[column] - content[column].mean()) / content[column].std()

X_scaled = norm[['area', 'bedrooms', 'bathrooms']].values
X_scaled = np.c_[X_scaled, np.ones(X_scaled.shape[0])]
y = norm["price"].values

#checking sizes of X and y to be sure that dot product is possible to be fulfilled
print("X_scaled shape:", X_scaled.shape)
print("y shape: ", y.shape)
print("X_scaled type is: ", type(X_scaled))
print("y type is: ", type(y))

X_scaled shape: (545, 4)
y shape:  (545,)
X_scaled type is:  <class 'numpy.ndarray'>
y type is:  <class 'numpy.ndarray'>


In [1733]:
# знайдіть параметри за допомогою аналітичного рішення;
w = np.linalg.inv(X_scaled.T@X_scaled) @ X_scaled.T @ y
print("w shape:", w.shape)
print("w type:", type(w),"\n")

#predicting y (price) to use it in MSE function 
y_predict = X_scaled @ w
print("y_predict shape: ", y_predict.shape)
print("y_predict type: ", type(y_predict),"\n")

def mse(y, y_predict):
   return np.mean((y - y_predict)**2)

print("Coefficients of linear regression using analytical formula: ")
w_dict= dict(zip(("area","bedrooms","bathrooms","Intercept"), map(float, w.flatten())))
for k,v in w_dict.items():
    print(f"{k}: {v}")
print("\n"+"MSE: ", mse(y, y_predict))


w shape: (4,)
w type: <class 'numpy.ndarray'> 

y_predict shape:  (545,)
y_predict type:  <class 'numpy.ndarray'> 

Coefficients of linear regression using analytical formula: 
area: 821968.5893534265
bedrooms: 300259.1646803202
bathrooms: 696447.7589857937
Intercept: 4766729.247706423

MSE:  1791170049977.319


In [1734]:
#напишіть функцію гіпотези лінійної регресії у векторному вигляді;
#створіть функцію для обчислення функції втрат у векторному вигляді;

def mse_func(X,y,w):    
    return np.linalg.norm(X @ w - y)**2 / len(X)

#derivative of the mse_func function
def grad(X, y, w):
    return 2 * X.T @ (X@w-y) / len(X)

In [1735]:
#реаліз градієнтного спуску;
# w was innitialized with zeros; 
#gamma = learning rate, max_iter = max amount of iterations, eps = accuracy
w = np.zeros(X_scaled.shape[1])
gamma = 1e-2
max_iter = 10000
eps = 1e-12



#calculating  current value of mse with random weights w
f_old = mse_func(X_scaled,y,w)
loss_history = [f_old]

#making one step to the side against gradient
w = w - gamma*grad(X_scaled,y,w)

#calculation of a new value of MSE error using above calculated weights w
f_new = mse_func(X_scaled,y,w)
loss_history.append(f_new)
# innitialisation of the first iteration 
i=1

#inside the loop we will calculate weights stepping against gradient while changing the mse function bigger than eps, or max_iter achieved
while np.abs(f_new - f_old) > eps and i < max_iter:
    w = w-gamma*grad(X_scaled,y,w)

    i +=1
    f_old = f_new
    f_new = mse_func(X_scaled,y,w)
    loss_history.append(f_new)

#after we quit the loop printing weights
print('Gradient Descent - w:', w,"\n")

losses = pd.DataFrame({
    "First 10 errors":[x.item() for x in loss_history[0:10]],
    "Middle 10 errors":[x.item() for x in loss_history[len(loss_history)//2-5 : len(loss_history)//2+5]],
    "Last 10 errors":[x.item() for x in loss_history[-10:]]
})

# predictions and MSE calculating
print("MSE:", mse_func(X_scaled, y, w))
losses.head(10)

Gradient Descent - w: [ 821968.60913693  300259.30870514  696447.60613195 4766729.24678282] 

MSE: 1791170049977.3474


,First 10 errors,Middle 10 errors,Last 10 errors
0,2.621383e+13,1.791170e+12,1.791170e+12
1,2.521931e+13,1.791170e+12,1.791170e+12
2,2.426582e+13,1.791170e+12,1.791170e+12
3,2.335163e+13,1.791170e+12,1.791170e+12
4,2.247510e+13,1.791170e+12,1.791170e+12
5,2.163465e+13,1.791170e+12,1.791170e+12
6,2.082877e+13,1.791170e+12,1.791170e+12
7,2.005601e+13,1.791170e+12,1.791170e+12
8,1.931499e+13,1.791170e+12,1.791170e+12
9,1.860439e+13,1.791170e+12,1.791170e+12


In [1736]:
#для перевірки спрогнозованих значень, використайте LinearRegression з бібліотеки scikit-learn 
#та порівняйте результати. 
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

#deleting last column with 1 to feed the model, because the model add this column automatically
X_lr = X_scaled[:, :-1]

regressor = LinearRegression().fit(X_lr, y)

print("Weights: ", regressor.coef_)
print("Intercept: ", regressor.intercept_, "\n")

print("Models score:", regressor.score(X_lr, y),"\n")

predictions = regressor.predict(X_lr)

print("MSE:", mean_squared_error(y, predictions), "\n")


Weights:  [821968.58935343 300259.16468032 696447.75898579]
Intercept:  4766729.247706422 

Models score: 0.4870830667058762 

MSE: 1791170049977.3193 



In [1737]:
from sklearn.linear_model import Ridge
from sklearn.metrics import mean_squared_error

X_ridge = X_scaled[:, :-1]
# Using Ridge Regression instead of Linear Regression
ridge_regressor = Ridge(alpha=0.1)  # alpha — это коэффициент регуляризации
ridge_regressor.fit(X_ridge, y)

print("Weights: ", ridge_regressor.coef_)
print("Intercept: ", ridge_regressor.intercept_, "\n")

# Prediction
predictions_scaled = ridge_regressor.predict(X_ridge)

# quality of model's prediction
print("Models score:", ridge_regressor.score(X_ridge, y),"\n")
print("MSE (Ridge Regression):", mean_squared_error(y_denorm, predictions), "\n")


Weights:  [821837.09979111 300262.77725515 696343.88898878]
Intercept:  4766729.247706422 

Models score: 0.4870830572845464 

MSE (Ridge Regression): 1791170049977.319 



In [1739]:
'''Выводы:
1. Расчитанные коэффициенты модели(веса модели) одинаковы, независимо от способа расчёта: 
градиент, аналитическая формула, LinearRegression, Ridge model.
И составляют:
area: 821968.5893534265
bedrooms: 300259.1646803202
bathrooms: 696447.7589857937
Intercept: 4766729.247706423
Интерпретировать данные веса модели можно следующим образом: 
1. даже без area, bedrooms, bathrooms базовая цена квартиры будет составлять величину Intercept: 4766729.247706423
2. Наибольшее влияние на цену имеет вес area, затем bathrooms, bedrooms, то есть за каждую дополнительную единицу указанного параметра 
цена будет меняться на размер веса данного параметра;
3. Ошибка предсказаний модели MSE составляет 1791170049977.3193, что говорит о том что модель ужаснo предсказывает данные.
Возможно линейные закономерности в этом случае недостаточной для предсказания зависимости и нужно использовать
Более сложную закономерность(полином).
4. Score модели составляет: 0.4870830667058762. Учитывая что score при идеальном предсказании равeн 1, говорит о том что
Модель очень слабо предсказывает разброс данных. 
5. При реализации градиента видно как изменяется функция ошибки(под соответствующим расчётам приведена таблица). Hачальное значение
Функции ошибки составляeт 2.621383e+13 и конечное значение составляет 1.791170e+12. 
Также видно что ошибка сильно замедляет снижение Примерно со средины количества итераций.
6. Также необходимо отметить что без нормализации данных(а именно столбца area) модель плохо предсказывает веса, так как порядок значений
В столбце area в 2000-2500 раз больше чем порядок значений в остальных столбцах('bedrooms', 'bathrooms').

PS: Tатьяна, отличие от вашего расчёта в том что у меня свободный коэффициент идёт первым затем все остальные, у вас наоборот'''

"Выводы:\n1. Расчитанные коэффициенты модели(веса модели) одинаковы, независимо от способа расчёта: \nградиент, аналитическая формула, LinearRegression, Ridge model.\nИ составляют:\narea: 821968.5893534265\nbedrooms: 300259.1646803202\nbathrooms: 696447.7589857937\nIntercept: 4766729.247706423\nИнтерпретировать данные веса модели можно следующим образом: \n1. даже без area, bedrooms, bathrooms базовая цена квартиры будет составлять величину Intercept: 4766729.247706423\n2. Наибольшее влияние на цену имеет вес area, затем bathrooms, bedrooms, то есть за каждую дополнительную единицу указанного параметра \nцена будет меняться на размер веса данного параметра;\n3. Ошибка предсказаний модели MSE составляет 1791170049977.3193, что говорит о том что модель ужаснo предсказывает данные.\nВозможно линейные закономерности в этом случае недостаточной для предсказания зависимости и нужно использовать\nБолее сложную закономерность(полином).\n4. Score модели составляет: 0.4870830667058762. Учитывая 